# Delta 对冲演示

本 Notebook 使用本项目中的 **真实** 模块 `cb_arb.delta_hedging.DeltaHedger`，对一条模拟的股票价格路径做日频 Delta 对冲，并展示组合价值变化。

数据生成方式与 `examples.run_simple_backtest` 中的 GBM 路径一致，保证可复现、可对照。

## 1. 路径设置与导入

导入 `DeltaHedger`、`HedgeState` 以及定价与参数模块。

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = os.getcwd()
project_root = cwd if os.path.isdir(os.path.join(cwd, 'src')) else os.path.abspath(os.path.join(cwd, '..'))
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from cb_arb.params import ConvertibleBondContract, TermStructure, CreditCurve
from cb_arb.delta_hedging import DeltaHedger, HedgeState

# 导入 Notebook 工具函数（用于保存输出文件）
notebooks_dir = os.path.join(project_root, 'notebooks')
if notebooks_dir not in sys.path:
    sys.path.insert(0, notebooks_dir)
from notebook_utils import save_figure, get_figures_dir, get_data_dir

print("项目根目录:", project_root)
print("src 目录已添加到 Python 路径:", src_dir)
print("输出目录:", os.path.join(project_root, 'output'))
print("DeltaHedger 导入成功。")

## 2. 生成股票价格路径（与 run_simple_backtest 相同公式）

使用 GBM 离散化公式生成日频路径，与示例脚本中的 `simulate_gbm_path` 逻辑一致。

In [ ]:
def simulate_gbm_path(S0, r, q, vol, dates, seed=42):
    dt = 1.0 / 252.0
    n = len(dates)
    rng = np.random.default_rng(seed)
    shocks = rng.normal(0.0, np.sqrt(dt), size=n)
    prices = [S0]
    for eps in shocks[1:]:
        s_prev = prices[-1]
        s_new = s_prev * np.exp((r - q - 0.5 * vol**2) * dt + vol * eps)
        prices.append(s_new)
    return pd.Series(prices, index=dates, name='stock')

dates = pd.date_range("2020-01-01", periods=250, freq="B")
S0, r, q, vol = 100.0, 0.02, 0.01, 0.25
stock_series = simulate_gbm_path(S0, r, q, vol, dates)

print("路径长度:", len(stock_series))
print("前 5 日股价:", stock_series.head().values)
print("后 5 日股价:", stock_series.tail().values)

## 3. 构建 DeltaHedger 并运行日频对冲

合约与期限结构与 `run_simple_backtest` 一致，使用真实 API `run_daily_hedging(stock_series)`。

In [ ]:
contract = ConvertibleBondContract(
    face_value=100.0,
    coupon_rate=0.03,
    maturity=3.0,
    conversion_ratio=1.0,
    issue_price=100.0,
    call_price=None,
    put_price=None,
    coupon_freq=2,
)
r_curve = TermStructure(rate_fn=lambda t: r)
q_curve = TermStructure(rate_fn=lambda t: q)
credit_curve = CreditCurve(spread_fn=lambda t: 0.03)
steps = 50
initial_cb_face = 100_000.0

hedger = DeltaHedger(
    contract=contract,
    r_curve=r_curve,
    q_curve=q_curve,
    credit_curve=credit_curve,
    vol=vol,
    steps=steps,
    initial_cb_face=initial_cb_face,
)

history = hedger.run_daily_hedging(stock_series)
print("对冲记录条数:", len(history))
print("首日 HedgeState: date=%s, cb_price=%.2f, delta=%.4f, hedge_shares=%.2f, portfolio_value=%.2f" % (
    history[0].date.date(), history[0].cb_price, history[0].cb_delta, history[0].hedge_shares, history[0].portfolio_value))
print("末日 HedgeState: date=%s, cb_price=%.2f, delta=%.4f, hedge_shares=%.2f, portfolio_value=%.2f" % (
    history[-1].date.date(), history[-1].cb_price, history[-1].cb_delta, history[-1].hedge_shares, history[-1].portfolio_value))

## 4. 组合价值与 Delta 随时间变化（真实输出）

将 `HedgeState` 列表转为 DataFrame，并绘图。

In [ ]:
hedge_df = pd.DataFrame([
    {
        'date': h.date,
        'stock_price': h.stock_price,
        'cb_price': h.cb_price,
        'cb_delta': h.cb_delta,
        'hedge_shares': h.hedge_shares,
        'portfolio_value': h.portfolio_value,
    }
 for h in history
])
hedge_df.set_index('date', inplace=True)

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
axes[0].plot(hedge_df.index, hedge_df['stock_price'], label='股价')
axes[0].set_ylabel('股价')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

axes[1].plot(hedge_df.index, hedge_df['cb_delta'], label='CB Delta', color='C1')
axes[1].set_ylabel('Delta')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

axes[2].plot(hedge_df.index, hedge_df['portfolio_value'], label='组合价值', color='C2')
axes[2].set_ylabel('组合价值')
axes[2].set_xlabel('日期')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
plt.suptitle('Delta 对冲模拟 (DeltaHedger.run_daily_hedging 真实输出)')
plt.tight_layout()

# 保存图片
save_figure(fig, '03_delta_hedging_simulation')

# 保存对冲历史数据
from datetime import datetime
data_dir = get_data_dir()
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_path = data_dir / f'hedge_history_{timestamp}.csv'
hedge_df.to_csv(csv_path, encoding='utf-8-sig')
print(f'对冲历史数据已保存到: {csv_path}')

plt.show()